# 처음 배우는 셸 스크립트#6

## 3부 예제와 함께 활용하는 셸 스크립트 활용
시스템 운영, 개발환경 구축, 클라우드 시스템 운영 시 발생하는 다양한 상황에서 셸 스크립트를 활용한다.  
상황 -> 방법 찾기 -> 스크립트 생성 -> 문제 해결  

## 9 환경 설정
여러 애플리케이션을 설치하다보면, 환경설정을 해야할 상황이 발생한다.  
예를 들어 클러스터된 서비스를 구축한다고 가정해보자. 클러스터간 ssh를 통해 패스워드 없이 ssh-key로 접속설정하거나  
클러스터간 시간 동기화를 위해 NTP 설정을 해야 하는 경우도 있다.  
소스 공유를 위해 NFS 스토리지에 마운트를 해야 하는 상황도 발생한다. 다 셸 스크립트로 할 수 있다

### 9.1 sshd 환경 설정 변경할 때
##### 상황 
보안을 위해 접근을 제한하거나, 포트를 변경하기도 한다. 모든 서버의 sshd 환경 설정을 동일하게 해야 한다.  
##### 방법 찾기
ssh에서 가장 설정 변경을 많이 하는 항목을 알아보고, 환경 설정파일의 위치와 해당 항목들의 설정값을 확인한다  
필요한 정보
- SSH 환경 설정파일 경로: /etc/ssh/sshd_config
- Port: SSH 기본 포트인 22번을 다른 번호로 변경할 때 주로 사용
- PermitRootLogin: root 계정으로 SSH 접근을 허용하는지 여부
- PasswordAuthentication: 패스워드를 이용한 인증을 허용할지에 대한 여부
- PubkeyAuthentication: 퍼블릭키를 이용한 인증을 허용할지에 대한 여부
프로세스
- 환경 설정파일 경로를 변수에 저장한다
- Switch~case문을 이용하여 해당 번호를 입력받으면 환경 설정을 한다
- 해당 경로에서 해당 항목을 찾아 sed를 이용하여 값을 변경하고, 파일에 적용한다
- 설정이 변경되었으면 SSH 서비스를 재시작한다
- 운영체제가 레드햇 리눅스이고, Port를 변경했다면 Selinux 설정을 변경한다
##### 스크립트 생성

In [ ]:
%%bash
#!/bin/bash

conf_path=/etc/ssh/sshd_config

function restart_system()
{
  echo "Restart sshd"
  systemctl restart sshd
}

function selinux()
{
  # 운영체제가 레드햇 리눅스이고, port를 수정했을 경우
  if [[ $(cat /etc/*release | grep -i redhat | wc -l) > 1 ]] && [[ $1 == 1 ]]
  then
    # SELinux에 해당 port 추가
    echo "Add port $port to selinux"
    semanage port -a -t ssh_port_t -p tcp $port
  fi
}

# 환경설정 파일 백업
cp $conf_path ${conf_path}.bak.$(date +%Y%m%d)

case $1 in
  1)
  # Port 변경
  read -p "Please input port: " port
  exist_conf=$(cat $conf_path | grep -e '^#Port' -e '^Port')
  sed -i "s/$exist_conf/Port $port/g" $conf_path
  restart_system
  selinux $1
  ;;
  2)
  # PermitRootLogin 변경
  read -p "Please input PermitRootLogin yes or no: " rootyn
  exist_conf=$(cat $conf_path | grep -e '^#PermitRootLogin' -e '^PermitRootLogin')
  sed -i "s/$exist_conf/PermitRootLogin $rootyn/g" $conf_path
  restart_system
  ;;
  3)
  # PasswordAuthentication 변경
  read -p "Please input PasswordAuthentication yes or no: " pwyn
  exist_conf=$(cat $conf_path | grep -e '^#PasswordAuthentication' -e '^PasswordAuthentication')
  sed -i "s/$exist_conf/PasswordAuthentication $pwyn/g" $conf_path
  restart_system
  ;;
  4)
  # PubkeyAuthentication 변경
  read -p "Please input PubkeyAuthentication yes or no: " keyyn
  exist_conf=$(cat $conf_path | grep -e '^#PubkeyAuthentication' -e '^PubkeyAuthentication')
  sed -i "s/$exist_conf/PubkeyAuthentication $keyyn/g" $conf_path
  restart_system
  ;;
  *)
  echo "Please input with following number"
  echo "1) Port  2) PermitRootLogin  3) PasswordAuthentication  4)PubkeyAuthentication"
  echo "Usage: config-sshd.sh 2"
esac



In [ ]:
%%bash
## 문제해결
bash conf-sshd.sh
bash conf.sshd.sh 1
bash conf.sshd.sh 2
bash conf.sshd.sh 3
bash conf.sshd.sh 4
cat /etc/ssh/sshd_config | grep -e '^Port' -e '^Permit' -e '^Password' -e '^Pubkey'
semanage port -l | grep ssh

### 9.2 ntp 서버 환경 설정할 때
##### 상황 
NTP 서버는 ntp를 설치하여 구축할 수도 있고, 보완하여 나온 chrony를 설치하여 구축할 수도 있다.  
NTP 패키지를 확인하고, 환경 설정을 동일하게 수정하자    
##### 방법 찾기
NTP에서 가장 설정 변경을 많이 하는 항목을 알아보고, 환경 설정파일의 위치와 해당 항목들의 설정값을 확인한다  
필요한 정보
- Chrony일 경우 환경 설정파일 경로: /etc/chrony.conf
- Ntp일 경우 환경 설정파일 경로: /etc/ntp.conf
- ntp server pool: ntp.conf와 chrony.cnf에서 동일하게 사용되는 ntp 서버 목록
- allow: ntp를 접근할 수 있는 네트워크 대역을 제한할 때 사용하며, chrony.conf에서 쓰임
- restrict: ntp를 접근할 수 있는 네트워크 대역을 제한할 때 사용하며, ntp.conf에서 쓰임
프로세스
- 파라미터로 ip 대역을 입력받아 정규 표현식에 의해 ip 대역인지를 확인한다
- 잘못 입력했다면 메시지를 출력한다
- 설치된 ntp 패키지 정보를 확인한다
- 기본으로 설정되어 있는 ntp 서버 풀을 주석 처리한다
- 주석 처리된 서버 풀 아래에 로컬 서버 정보를 서버 풀로 추가한다
- IP 대역을 확인 후 있으면 allow나 restrict로 설정한다
- Ntp 서비스를 재시작한다
- Firewall에 ntp 포트를 추가한다
##### 스크립트 생성

In [ ]:
%%bash
#!/bin/bash

ip=""
netmask=""
conf=""
service=""

# IP CIDR을 NetMask로 변경함.
function transfer_iprange()
{
    ip=${1%/*}  # 매개변수 확장자 %으로 /이후의 글자를 날려버린다
    if [[ ${1#*/} == 16 ]]; then netmask="255.255.0.0"; fi  # 매개변수 확장자 #으로 /이전의 글자를 날려버린다
    if [[ ${1#*/} == 23 ]]; then netmask="255.255.254.0"; fi
    if [[ ${1#*/} == 24 ]]; then netmask="255.255.255.0"; fi
    if [[ ${1#*/} == 28 ]]; then netmask="255.255.240.0"; fi
}

if [[ -n $1 ]]; then
  # 정규 표현식을 이용하여 IP 범위를 정상적으로 입력했는지 확인
  range_chk=$(echo "$1" | grep -E "^[0-9]{1,3}.[0-9]{1,3}.[0-9]{1,3}.0/[0-9]{2}$" | wc -l)
  # 정규 표현식과 다르다면 메시지를 출력하고 스크립트 종료
  if [[ range_chk -eq 0 ]]; then
    echo "This ip cidr is wrong. Please input the right ip cidr."
    exit;
  fi
fi

# chrony가 설치되어 있는지 ntp가 설치되어 있는지 환경설정 파일을 통해 확인
if [[ -f /etc/chrony.conf ]]; then 
  conf=/etc/chrony.conf
  service=chronyd.service
elif [[ -f /etc/ntp.conf ]]; then 
  conf=/etc/ntp.conf
  service=ntpd.service
fi

# 서버 주소 변경
sed -i "s/^server/#server/g" $conf
sed -i "/^#server 3/ a \server 127.127.1.0" $conf
  
# 파라메터로 IP가 있으면  allow 설정
if [[ -n $1 && -f /etc/chrony.conf ]]; then
    sed -i "/^#allow/ a \allow $1" $conf
# 환경설정 파일이 ntp.conf 일 경우
elif [[ -n $1 && -f /etc/ntp.conf ]]; then
    transfer_iprange $1
    restrict="restrict $ip mask $netmask nomodify notrap"
    sed -i "/^#restrict/ a \restrict $restrict" $conf
fi

# 서비스 재시작  
echo "systemctl restart $service"
systemctl restart $service

# 포트 추가
echo "firewall-cmd --add-service=ntp"
firewall-cmd --add-service=ntp
echo "firewall-cmd --add-service=ntp --permanant"
firewall-cmd --add-service=ntp --permanant


In [ ]:
%%bash
## 문제해결
bash conf-ntp.sh
bash conf-ntp.sh 10.10.0.10/24
bash conf-ntp.sh 10.10.0.0/24
cat /etc/chrony.conf | grep -e server -e allow

### 9.3 lvm 환경 설정할 때 (생략)
### 9.4 NFS 스토리지 마운트할 때
##### 상황 
NFS 스토리지는 네트워크를 통해 볼륨을 사용할 수 있으며, 여러 서버에서 동시에 접근하여 볼륨을 사용할 수 있는 공유 스토리지이다.  
개발 환경을 구축할 때, 데이터베이스 시스템을 구축할 때 많이 사용된다.  
NFS 스토리지를 리눅스에서 사용할 경우, 스토리지와 마운트할 디렉터리를 생성해야 한다.  
재부팅 시에도 스토리지 연결을 유지하기 위해 /etc/fstab에 NFS 스토리지 정보를 등록해야 한다.  
셸 스크립트로 만들어두면 필요시 유용하게 NFS 스토리지를 마운트하여 사용할 수 있다.  
##### 방법 찾기
NFS 스토리지 마운트는 베어메탈 환경의 서버, 가상머신 환경의 서버에서도 사용할 수 있다.
필요한 정보
- 마운트 명령어: mount
- 마운트할 대상 NFS 서버 경로
- 마운트할 디렉터리
- 마운트할 NFS 버전 및 옵션
프로세스
- 마운트할 대상 NFS 서버 경로를 변수에 저장한다
- 마운트할 디렉터리명은 변수에 저장한다
- 마운트할 디렉터리가 있는지 체크 후 디렉터리를 생성한다
- 생성한 디렉터리에 마운트 대상 NFS를 기본 옵션으로 마운트한다
- 마운트가 되면 mount 명령어를 이용하여 마운트된 디렉터리의 NFS 정보를 확인한다
- /etc/fstab에 해당 정보를 추가한다
- /etc/fstab을 열어 추가된 정보를 확인한다
##### 스크립트 생성

In [ ]:
%%bash
#!/bin/bash

# 변수에 마운트 대상 NFS 경로 및 디렉토리 저장 
nfs_server="nfs.host01:/temp"
nfs_dir=/nfs_temp

# 마운트할 디렉토리가 있는지 체크후 없으면 디렉토리 생성
if [ ! -d $nfs_dir ]; then mkdir -p $nfs_dir; fi

# 해당 NFS와 디렉토리 마운트
mount -t nfs $nfs_server $nfs_dir

# 마운트 정보에서 마운트 타입과 옵션 추출
nfs_type=$(mount | grep $nfs_dir | awk '{print $5}')
nfs_opt=$(mount | grep $nfs_dir | awk '{print $6}' | awk -F ',' '{print $1","$2","$3}')

# 추출한 마운트 정보를 조합하여 /etc/fstab에 설정
echo "$nfs_server  $nfs_dir  $nfs_type  ${nfs_opt:1}  1  1" >> /etc/fstab

# 설정한 /etc/fstab 내용 확인
cat /etc/fstab | grep $nfs_dir 

# 마운트 된 디렉토리 정보 확인
df -h | grep $nfs_dir


In [ ]:
%%bash
##문제 해결
mount | grep /nfs_temp
mount | grep /nfs_temp | awk '{print $5}'
mount | grep nfs_dir | awk '{print $6}' | awk -F ',' ' {print $1","$2","$3}'
bash conf-nfs.sh

### 9.5 네트워크 ip 설정할 때
##### 상황 
엔지니어는 쉽지만 일반인은 ip설정이 매우 어려운 일일 수 있다.  
처음 네트워크ip를 설정할 때 스크립트화해 놓는다면 좋다.  
##### 방법 찾기
리눅스 종류와 버전에 따라 다르다. 데비안은 우분투 18.04이상, 페도라는 레드햇 8, centos8이상에서 설정한다.  
필요한 정보
- 데비안 계열의 네트워크 설정 방법: netplan 파일 설정
- 페도라 계열의 네트워크 설정 방법: nmcli 명령어로 설정
- network interface name
- ip/cidr
- gateway
- dns
프로세스
- 운영체제 타입을 확인 후 변수에 저장한다
- 네트워크 디바이스 명을 조회하여 보여준다
- 네트워크 IP 설정에 필요한 정보를 사용자로부터 입력받는다
- 네트워크 정보를 입력받지 않았을 경우 입력하라는 메시지 출력 후 스크립트를 종료한다
- 운영체제 타입이 페도라일 경우 nmcli를 이용해 네트워크 IP 설정한다
- 운영체제 타입이 데비안인 경우 netplan 파일을 생성하여 IP 설정한다

##### 스크립트 생성

In [ ]:
%%bash
#!/bin/bash

# 운영체제 타입 확인
ostype=$(cat /etc/*release| grep ID_LIKE | sed "s/ID_LIKE=//;s/\"//g")

# 네트워크 정보를 사용자로부터 입력 받음
echo "=== Network Devices ==="
ip a | grep '^[0-9]' | awk '{print $1" "$2}' | grep -v -e 'lo' -e 'v' -e 't'
read -p "Please input network interface: " net_name
read -p "Please input network ip(ex:192.168.122.10/24): " net_ip
read -p "Please input network gateway: " net_gw
read -p "Please input network dns: " net_dns

# 하나라도 입력하지 않았을 경우 입력하라는 메시지 출력후 스크립트 종료
if [[ -z $net_name ]] || [[ -z $net_ip ]] || [[ -z $net_gw ]] || [[ -z $net_dns ]]; then
  echo "You need to input network information. Please retry this script" 
  exit;
fi 

# 운영체제가 페도라 계열일 경우 nmcli 명령어를 이용하여 네트워크 IP 설정
if [[ $ostype == "fedora" ]]; then
  nmcli con add con-name $net_name type ethernet ifname $net_name ipv4.address $net_ip ipv4.gateway $net_gw ipv4.dns $net_dns ipv4.method manual
  nmcli con up $net_name
# 운영체제가 데미안 계열일 경우 netplan에 yaml 파일을 생성하여 네트워크 IP 설정
elif [[ $ostype == "debian" ]]; then
  ip_chk=$(grep $net_name /etc/netplan/*.yaml|wc -l)
  # 설정하고자 하는 IP로 설정파일이 없을 경우 관련 네트워크 yaml 파일 생성
  if [ $ip_chk -eq 0 ]; then                      #EOF 다음 라인부터 EOF가 나올 떄까지의 모든 문자열을 .yaml파일에 저장한다는 의미다
    cat > /etc/netplan/${net_name}.yaml << EOF  
network:
 version: 2
 renderer: networkd
 ethernets:
   $net_name:
     dhcp4: no
     dhcp6: no
     addresses: [$net_ip]
     gateway4: $net_gw
     nameservers:
       addresses: [$net_dns]
EOF
    echo "cat /etc/netplan/${net_name}.yaml"
    cat /etc/netplan/${net_name}.yaml
    echo "apply netplan"
    netplan apply
  else
    echo "This $net_name is configured already." 
  fi
fi



In [ ]:
%%bash
##문제 해결
bash conf-netip.sh
ip a s